In [1]:
## Slimming down .csv

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US_AQI.csv')

In [2]:
df['Defining Parameter'].value_counts()/len(df)*100

Ozone    53.935352
PM2.5    25.382313
PM10      8.249015
NO2       6.393897
CO        6.039423
Name: Defining Parameter, dtype: float64

In [3]:
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

In [4]:
df['Date'].dt.year

0          2022
1          2022
2          2022
3          2022
4          2022
           ... 
5617320    1980
5617321    1980
5617322    1980
5617323    1980
5617324    1980
Name: Date, Length: 5617325, dtype: int64

In [5]:
## Choosing this size for the speed of the app. It can be adjusted easily.

df2 = df[df['Date'].dt.year >= 2016]

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1034119 entries, 0 to 1034118
Data columns (total 15 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Unnamed: 0                 1034119 non-null  int64         
 1   CBSA Code                  1034119 non-null  int64         
 2   Date                       1034119 non-null  datetime64[ns]
 3   AQI                        1034119 non-null  int64         
 4   Category                   1034119 non-null  object        
 5   Defining Parameter         1034119 non-null  object        
 6   Number of Sites Reporting  1034119 non-null  int64         
 7   city_ascii                 1034119 non-null  object        
 8   state_id                   1034119 non-null  object        
 9   state_name                 1034119 non-null  object        
 10  lat                        1034119 non-null  float64       
 11  lng                        1034119 no

In [7]:
df2 = df2.drop(columns = ['Unnamed: 0', 'timezone', 'CBSA Code', 'Number of Sites Reporting'])
df2 = df2.rename(columns = {'city_ascii':'city', 'state_name': 'state'})

In [8]:
df2['city_full'] = df2['city']+', '+df2['state_id']

In [9]:
## Correlation, first example.

grouped = df2.groupby(['city_full']).filter(lambda x: all(['Ozone' in x['Defining Parameter'].values, 'PM2.5' in x['Defining Parameter'].values]))
grouped.groupby(['city_full', 'Defining Parameter']).agg({'AQI': 'mean'})
# pivot the table to get Ozone and PM2.5 AQI values in separate columns
grouped = grouped.pivot_table(index='city_full', columns='Defining Parameter')

# compute correlation between Ozone and PM2.5 AQI values
corr = np.corrcoef(grouped['AQI']['Ozone'], grouped['AQI']['PM2.5'])[0][1]

print(f"The correlation between AQI for Ozone and PM2.5 is {corr:.2f}")

The correlation between AQI for Ozone and PM2.5 is 0.59


In [10]:
from plotnine import *
from itertools import combinations

In [11]:
## All pairwise correlations

# get unique categories in Defining Parameter
categories = df2['Defining Parameter'].unique()

# iterate over all pairs of categories in Defining Parameter
corr_df = pd.DataFrame(columns=['Category 1', 'Category 2', 'Correlation'])
for pair in combinations(categories, 2):
    # filter the dataframe
    grouped = df2.groupby(['city_full']).filter(lambda x: all([pair[0] in x['Defining Parameter'].values, pair[1] in x['Defining Parameter'].values]))

    # group by city_full and Defining Parameter and compute mean AQI
    grouped = grouped.groupby(['city_full', 'Defining Parameter']).agg({'AQI': 'mean'})
    grouped = grouped.pivot_table(index='city_full', columns='Defining Parameter')
    
    # compute correlation between AQI for Ozone and PM2.5
    corr = np.corrcoef(grouped['AQI'][pair[0]], grouped['AQI'][pair[1]])[0][1]
    corr_df = corr_df.append({'Category 1': pair[0], 'Category 2': pair[1], 'Correlation': corr}, ignore_index=True)

print(corr_df)

  Category 1 Category 2 Correlation
0      PM2.5      Ozone    0.591928
1      PM2.5        NO2    0.414913
2      PM2.5       PM10    0.523377
3      PM2.5         CO     0.00549
4      Ozone        NO2     0.51582
5      Ozone       PM10    0.462006
6      Ozone         CO   -0.153611
7        NO2       PM10     0.52529
8        NO2         CO   -0.072628
9       PM10         CO   -0.008876


In [12]:
## Correlation is the same between PM2.5 and Ozone.

df2.to_csv('US_AQI(2016-2022).csv', index=False)